# Candidate Extraction : Diseases

This notebook is meant for in-house demonstration of candidate extraction and featurization of tables. It assumes an input file in XHTML format, a strict form of HTML that coincides with XML structure, allowing for easy display (HTML) and safe tree traversal (XML).

In [ ]:
%load_ext autoreload
%autoreload 2

### Candidate Extraction

First, import the 'HTMLParser' class to read HTML tables

In [39]:
from snorkel.parser import HTMLParser
html_parser = HTMLParser(path='data/diseases/diseases.xhtml')

The "TableParser" class divides the html doc into cells, adding a 'cell_id' attribute to each cell for future traversal, and creating "Cell" objects that have attributes such as row number, column number, html tag, html attributes, and any tags/attributes on a cells ancestors in the table.

In [41]:
from snorkel.parser import TableParser
table_parser = TableParser()

As usual, pass these to a Corpus object for digestion.

In [42]:
# from snorkel.parser import Corpus
# %time corpus = Corpus(html_parser, table_parser)

from snorkel.parser import CorpusParser
cp = CorpusParser(html_parser, table_parser)
%time corpus = cp.parse_corpus(name='Diseases Corpus')

CPU times: user 140 ms, sys: 17.9 ms, total: 157 ms
Wall time: 211 ms


In [43]:
doc = corpus.documents[0]
print doc
print doc.tables[0]
print doc.tables[0].cells[0]
print doc.tables[0].cells[0].phrases[0]

Document('0', Corpus (Diseases Corpus))
Table('0', 0)
Cell('0', 0, 0, 'Disease')
Phrase('0', 0, 0, 0, u'Disease')


Load the good 'ole disease dictionary for recognizing disease names.

In [44]:
from load_dictionaries import load_disease_dictionary

# Load the disease phrase dictionary
diseases = load_disease_dictionary()
print "Loaded %s disease phrases!" % len(diseases)

Loaded 507899 disease phrases!


Here we use a new CandidateSpace object, CellNgrams. It inherits from Ngrams, and ensures that the Table context object is broken up into cells before being passed into the usual routine for pulling out Ngrams.

In [45]:
from snorkel.candidates import TableNgrams
from snorkel.matchers import DictionaryMatch

# Define a candidate space
ngrams = TableNgrams(n_max=3)

# Define a matcher
disease_matcher = DictionaryMatch(d=diseases, longest_match_only=False)

Passing the CandidateSpace, Matcher, and Context objects to a Candidates object, extraction is performed, and we see that a number of disease CellNgrams are returned.

In [46]:
# With new Candidates object:
# from snorkel.candidates import Candidates
# %time candidates = Candidates(table_ngrams, disease_matcher, corpus.get_contexts())

# With old Candidates object:
from snorkel.candidates import CandidateExtractor
ce = CandidateExtractor(ngrams, disease_matcher)
%time candidates = ce.extract(corpus.get_tables(), name='all')

for cand in candidates: print cand

CPU times: user 55.9 ms, sys: 3.62 ms, total: 59.5 ms
Wall time: 57.5 ms
Ngram("Disease", context=None, chars=[0,6], words=[0,0])
Ngram("Location", context=None, chars=[0,7], words=[0,0])
Ngram("Polio", context=None, chars=[0,4], words=[0,0])
Ngram("Chicken Pox", context=None, chars=[0,10], words=[0,1])
Ngram("plague", context=None, chars=[31,36], words=[3,3])
Ngram("Scurvy", context=None, chars=[0,5], words=[0,0])
Ngram("Problem", context=None, chars=[0,6], words=[0,0])
Ngram("Arthritis", context=None, chars=[0,8], words=[0,0])
Ngram("Yellow Fever", context=None, chars=[0,11], words=[0,1])
Ngram("Fever", context=None, chars=[7,11], words=[1,1])
Ngram("Hypochondria", context=None, chars=[0,11], words=[0,0])
Ngram("Disease", context=None, chars=[0,6], words=[0,0])
Ngram("Location", context=None, chars=[0,7], words=[0,0])
Ngram("Polio", context=None, chars=[0,4], words=[0,0])
Ngram("Chicken Pox", context=None, chars=[0,10], words=[0,1])
Ngram("plague", context=None, chars=[31,36], words=

### Feature Generation

We can then generate features on our set of candidates, including *new and improved* table features!

In [47]:
from snorkel.features import TableNgramFeaturizer
featurizer = TableNgramFeaturizer(candidates, corpus)

Extracting features...
Extracted 131 features for each of 22 candidates


In [48]:
featurizer.get_features_by_id(candidates[0].id)

[u'BASIC_NGRAM_Disease',
 'TABLE_ROW_NUM_0',
 'TABLE_COL_NUM_1',
 'TABLE_HTML_TAG_th',
 'TABLE_HTML_ANC_TAG_tr',
 'TABLE_HTML_ANC_TAG_tbody',
 'TABLE_HTML_ANC_TAG_table',
 'TABLE_HTML_ANC_TAG_body',
 'TABLE_HTML_ANC_ATTR_align=left',
 'TABLE_HTML_ANC_ATTR_size=5',
 'TABLE_HTML_ANC_ATTR_font=blue',
 u'TABLE_ROW_NGRAM_Disease',
 u'TABLE_ROW_NGRAM_Location',
 u'TABLE_ROW_NGRAM_Year',
 u'TABLE_COL_NGRAM_Disease',
 u'TABLE_COL_NGRAM_Polio',
 u'TABLE_COL_NGRAM_Chicken',
 u'TABLE_COL_NGRAM_Chicken_Pox',
 u'TABLE_COL_NGRAM_Chicken_Pox_are',
 u'TABLE_COL_NGRAM_Pox',
 u'TABLE_COL_NGRAM_Pox_are',
 u'TABLE_COL_NGRAM_Pox_are_bad',
 u'TABLE_COL_NGRAM_are',
 u'TABLE_COL_NGRAM_are_bad',
 u'TABLE_COL_NGRAM_are_bad_.',
 u'TABLE_COL_NGRAM_bad',
 u'TABLE_COL_NGRAM_bad_.',
 u'TABLE_COL_NGRAM_.',
 u'TABLE_COL_NGRAM_So',
 u'TABLE_COL_NGRAM_So_is',
 u'TABLE_COL_NGRAM_So_is_the',
 u'TABLE_COL_NGRAM_is',
 u'TABLE_COL_NGRAM_is_the',
 u'TABLE_COL_NGRAM_is_the_plague',
 u'TABLE_COL_NGRAM_the',
 u'TABLE_COL_NGRAM_t

Ta-da! Next up: feeding these features into the learning machine.